# 🏭 Hackathon - Détection de Défauts Industriels avec IA

## 📋 Objectif
Construire un système complet de détection de défauts sur des pièces industrielles en utilisant:
- **Machine Learning / Deep Learning** pour la classification
- **CBIR** (Content-Based Image Retrieval) pour la recherche par similarité
- **VLM** (Vision Language Model) pour la description automatique

---

## 🗓️ Programme
| Jour | Thème | Livrables |
|------|-------|----------|
| 1 | Fondamentaux | Pipeline ML/DL baseline |
| 2 | Optimisation | Hyperparamètres + Interface Streamlit |
| 3 | CBIR | Système de recherche par similarité |
| 4 | VLM | Description automatique + Intégration finale |

---

## ⚠️ Règles du Hackathon
- ✅ Utiliser les ressources fournies
- ✅ Collaborer en équipe
- ✅ Documenter votre code
- ❌ Ne pas copier le code final des autres équipes

**Bonne chance! 🚀**

---
# 📦 SECTION 0: Configuration de l'Environnement
---

In [ ]:
# Installation des dépendances (décommenter si nécessaire)
# !pip install torch torchvision scikit-learn xgboost lightgbm
# !pip install matplotlib seaborn pillow
# !pip install transformers accelerate  # Pour VLM

In [ ]:
# Imports de base
import os
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
from PIL import Image
from pathlib import Path
import warnings
warnings.filterwarnings('ignore')

# Deep Learning
import torch
import torch.nn as nn
from torchvision import models, transforms
from torch.utils.data import DataLoader, Dataset

# Machine Learning
from sklearn.model_selection import train_test_split, GridSearchCV
from sklearn.preprocessing import StandardScaler
from sklearn.svm import SVC
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import (
    accuracy_score, precision_score, recall_score, f1_score,
    confusion_matrix, classification_report
)

print(f"PyTorch version: {torch.__version__}")
print(f"CUDA available: {torch.cuda.is_available()}")
DEVICE = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
print(f"Using device: {DEVICE}")

In [ ]:
# Configuration des chemins - À MODIFIER selon votre structure
DATA_DIR = Path("./data")
TRAIN_DIR = DATA_DIR / "train"
TEST_DIR = DATA_DIR / "test"
MODELS_DIR = Path("./models")
MODELS_DIR.mkdir(exist_ok=True)

# Classes
CLASSES = ['non_defective', 'defective']
NUM_CLASSES = 2

# Paramètres images
IMG_SIZE = 224
BATCH_SIZE = 32

---
# 📊 SECTION 1: Exploration des Données (EDA)
---

## 💡 Objectif
Comprendre la distribution de vos données avant de construire le modèle.

## 🎯 TODO pour votre équipe:
1. Compter le nombre d'images par classe
2. Visualiser quelques exemples
3. Vérifier la taille et qualité des images

In [ ]:
# TODO: Compléter cette fonction pour compter les images
def count_images(directory):
    """
    Compte le nombre d'images par classe dans un répertoire.
    
    Args:
        directory: Chemin vers le répertoire (train ou test)
    
    Returns:
        dict: {'defective': count, 'non_defective': count}
    """
    counts = {}
    
    # VOTRE CODE ICI
    # Hint: Parcourir les sous-dossiers et compter les fichiers .jpg, .png
    
    return counts

# Test
print("Distribution Train:", count_images(TRAIN_DIR))
print("Distribution Test:", count_images(TEST_DIR))

In [ ]:
# TODO: Visualiser des exemples d'images
def visualize_samples(directory, n_samples=4):
    """
    Affiche des exemples d'images pour chaque classe.
    
    Hint: Utilisez matplotlib avec subplot
    """
    fig, axes = plt.subplots(2, n_samples, figsize=(15, 6))
    
    for i, class_name in enumerate(CLASSES):
        class_dir = directory / class_name
        # VOTRE CODE ICI
        # 1. Lister les fichiers dans class_dir
        # 2. Charger n_samples images
        # 3. Afficher avec axes[i, j].imshow()
        pass
    
    plt.tight_layout()
    plt.show()

visualize_samples(TRAIN_DIR)

### ❓ Questions de réflexion:
1. Vos classes sont-elles équilibrées?
2. Quelles caractéristiques visuelles distinguent les défauts?
3. Y a-t-il des variations de luminosité/angle dans les images?

---
# 🖼️ SECTION 2: Prétraitement des Images
---

## 💡 Concepts clés

### Normalisation ImageNet
Les modèles pré-entraînés (VGG, ResNet) attendent des images normalisées:
- **Mean**: [0.485, 0.456, 0.406]
- **Std**: [0.229, 0.224, 0.225]

### Data Augmentation
Techniques pour augmenter artificiellement le dataset:
- Rotation, flip horizontal
- Variation de luminosité
- Crop aléatoire

In [ ]:
# Transforms pour l'entraînement (avec augmentation)
train_transform = transforms.Compose([
    transforms.Resize((IMG_SIZE, IMG_SIZE)),
    transforms.RandomHorizontalFlip(p=0.5),
    transforms.RandomRotation(degrees=15),
    transforms.ColorJitter(brightness=0.2, contrast=0.2),
    transforms.ToTensor(),
    transforms.Normalize(
        mean=[0.485, 0.456, 0.406],
        std=[0.229, 0.224, 0.225]
    )
])

# Transforms pour test/inférence (PAS d'augmentation)
test_transform = transforms.Compose([
    transforms.Resize((IMG_SIZE, IMG_SIZE)),
    transforms.ToTensor(),
    transforms.Normalize(
        mean=[0.485, 0.456, 0.406],
        std=[0.229, 0.224, 0.225]
    )
])

print("✅ Transforms définis")

In [ ]:
# TODO: Créer un Dataset personnalisé
class DefectDataset(Dataset):
    """
    Dataset pour les images de défauts.
    
    Structure attendue:
    data_dir/
    ├── defective/
    │   └── *.jpg, *.png
    └── non_defective/
        └── *.jpg, *.png
    """
    
    def __init__(self, data_dir, transform=None):
        self.data_dir = Path(data_dir)
        self.transform = transform
        self.samples = []  # Liste de (chemin_image, label)
        
        # TODO: Remplir self.samples
        # Hint: Parcourir chaque classe, chaque image
        # Label: 0 = non_defective, 1 = defective
        
        # VOTRE CODE ICI
        
    def __len__(self):
        return len(self.samples)
    
    def __getitem__(self, idx):
        img_path, label = self.samples[idx]
        
        # TODO: Charger et transformer l'image
        # Hint: Image.open(), self.transform()
        
        # VOTRE CODE ICI
        image = None  # À remplacer
        
        return image, label

# Test
# train_dataset = DefectDataset(TRAIN_DIR, transform=train_transform)
# print(f"Nombre d'images: {len(train_dataset)}")

---
# 🧠 SECTION 3: Extraction de Caractéristiques avec CNN
---

## 💡 Concept: Transfer Learning

Utiliser un CNN pré-entraîné sur ImageNet comme **extracteur de features**:
1. Charger le modèle (VGG16, ResNet50, etc.)
2. Retirer la dernière couche (classification)
3. Passer l'image → Obtenir un vecteur de features

### Dimensions des features par modèle:
| Modèle | Dimension |
|--------|----------|
| VGG16 | 4096 |
| ResNet50 | 2048 |
| DenseNet121 | 1024 |
| MobileNetV2 | 1280 |

In [ ]:
# Exemple: Feature Extractor avec ResNet50
class FeatureExtractor(nn.Module):
    """
    Extrait les features d'une image en utilisant un CNN pré-entraîné.
    """
    
    def __init__(self, model_name='resnet50'):
        super().__init__()
        
        if model_name == 'resnet50':
            base_model = models.resnet50(pretrained=True)
            # Retirer la dernière couche FC
            self.features = nn.Sequential(*list(base_model.children())[:-1])
            self.output_dim = 2048
            
        elif model_name == 'vgg16':
            base_model = models.vgg16(pretrained=True)
            self.features = base_model.features
            self.avgpool = base_model.avgpool
            # Utiliser seulement les features, pas le classifier
            self.output_dim = 512 * 7 * 7  # 25088
            
        # TODO: Ajouter d'autres modèles (DenseNet, MobileNet)
        
        # Geler les poids (pas de fine-tuning)
        for param in self.parameters():
            param.requires_grad = False
    
    def forward(self, x):
        x = self.features(x)
        x = x.view(x.size(0), -1)  # Flatten
        return x

# Test
extractor = FeatureExtractor('resnet50').to(DEVICE)
extractor.eval()

# Test avec une image aléatoire
dummy_img = torch.randn(1, 3, 224, 224).to(DEVICE)
features = extractor(dummy_img)
print(f"Shape des features: {features.shape}")

In [ ]:
# TODO: Extraire les features de tout le dataset
def extract_all_features(data_dir, extractor, transform):
    """
    Extrait les features de toutes les images d'un répertoire.
    
    Returns:
        features: np.array de shape (n_samples, feature_dim)
        labels: np.array de shape (n_samples,)
        paths: liste des chemins d'images
    """
    features_list = []
    labels_list = []
    paths_list = []
    
    extractor.eval()
    
    # TODO: Parcourir les images et extraire les features
    # Hint: 
    # 1. Charger l'image avec PIL
    # 2. Appliquer transform
    # 3. Passer dans extractor
    # 4. Ajouter aux listes
    
    # VOTRE CODE ICI
    
    return np.array(features_list), np.array(labels_list), paths_list

# Extraction
# X_train, y_train, paths_train = extract_all_features(TRAIN_DIR, extractor, test_transform)
# print(f"Features train: {X_train.shape}")

---
# 🧠 SECTION 3b: Entraînement CNN Baseline (Deep Learning)
---

## 💡 Objectif
Entraîner un CNN from scratch pour comparer avec l'approche hybride (features pré-entraînées + shallow classifiers).

### Architecture suggérée:
```
Input (224×224×3)
    ↓
Conv2D(32) → BatchNorm → ReLU → MaxPool
    ↓
Conv2D(64) → BatchNorm → ReLU → MaxPool
    ↓
Conv2D(128) → BatchNorm → ReLU → MaxPool
    ↓
Conv2D(256) → BatchNorm → ReLU → MaxPool
    ↓
Global Average Pooling
    ↓
FC(256) → ReLU → Dropout
    ↓
FC(2) → Softmax
```

In [ ]:
# TODO: Définir le CNN Baseline
class BaselineCNN(nn.Module):
    """
    CNN personnalisé pour la classification de défauts.
    """
    
    def __init__(self, num_classes=2):
        super().__init__()
        
        # TODO: Définir les couches convolutionnelles
        # Hint: nn.Conv2d(in_channels, out_channels, kernel_size)
        # Hint: nn.BatchNorm2d(num_features)
        # Hint: nn.MaxPool2d(kernel_size)
        
        self.features = nn.Sequential(
            # Block 1: 3 → 32
            # VOTRE CODE ICI
            
            # Block 2: 32 → 64
            # VOTRE CODE ICI
            
            # Block 3: 64 → 128
            # VOTRE CODE ICI
            
            # Block 4: 128 → 256
            # VOTRE CODE ICI
        )
        
        # Global Average Pooling
        self.avgpool = nn.AdaptiveAvgPool2d((1, 1))
        
        # Classifier
        self.classifier = nn.Sequential(
            nn.Linear(256, 256),
            nn.ReLU(inplace=True),
            nn.Dropout(0.5),
            nn.Linear(256, num_classes)
        )
    
    def forward(self, x):
        x = self.features(x)
        x = self.avgpool(x)
        x = x.view(x.size(0), -1)
        x = self.classifier(x)
        return x

# Test
# model = BaselineCNN().to(DEVICE)
# dummy = torch.randn(1, 3, 224, 224).to(DEVICE)
# output = model(dummy)
# print(f"Output shape: {output.shape}")  # Devrait être (1, 2)

In [ ]:
# TODO: Fonction d'entraînement du CNN
def train_cnn(model, train_loader, val_loader, epochs=20, lr=0.001):
    """
    Entraîne le CNN avec early stopping.
    
    Args:
        model: Le CNN à entraîner
        train_loader: DataLoader d'entraînement
        val_loader: DataLoader de validation
        epochs: Nombre d'époques maximum
        lr: Learning rate
    
    Returns:
        model: Le modèle entraîné
        history: Dict avec les métriques par époque
    """
    criterion = nn.CrossEntropyLoss()
    optimizer = torch.optim.Adam(model.parameters(), lr=lr)
    scheduler = torch.optim.lr_scheduler.ReduceLROnPlateau(
        optimizer, mode='min', factor=0.5, patience=3
    )
    
    history = {'train_loss': [], 'val_loss': [], 'val_acc': []}
    best_val_loss = float('inf')
    patience_counter = 0
    
    for epoch in range(epochs):
        # TODO: Implémenter la boucle d'entraînement
        # 1. model.train()
        # 2. Pour chaque batch: forward → loss → backward → step
        # 3. model.eval() pour validation
        # 4. Calculer val_loss et val_acc
        # 5. Early stopping si pas d'amélioration
        
        # VOTRE CODE ICI
        
        print(f"Epoch {epoch+1}/{epochs}")
    
    return model, history

# Entraînement
# cnn_model = BaselineCNN().to(DEVICE)
# cnn_model, history = train_cnn(cnn_model, train_loader, val_loader, epochs=20)

In [ ]:
# TODO: Visualiser les courbes d'apprentissage
def plot_training_history(history):
    """
    Affiche les courbes de loss et accuracy.
    """
    fig, (ax1, ax2) = plt.subplots(1, 2, figsize=(12, 4))
    
    # Loss
    ax1.plot(history['train_loss'], label='Train Loss')
    ax1.plot(history['val_loss'], label='Val Loss')
    ax1.set_xlabel('Epoch')
    ax1.set_ylabel('Loss')
    ax1.legend()
    ax1.set_title('Training & Validation Loss')
    
    # Accuracy
    ax2.plot(history['val_acc'], label='Val Accuracy', color='green')
    ax2.set_xlabel('Epoch')
    ax2.set_ylabel('Accuracy')
    ax2.legend()
    ax2.set_title('Validation Accuracy')
    
    plt.tight_layout()
    plt.show()

# plot_training_history(history)

### 💡 Optimisation du CNN (si nécessaire)

Si les performances ne sont pas satisfaisantes:
1. **Data Augmentation** plus agressive
2. **Learning Rate Scheduler** (ReduceLROnPlateau)
3. **Regularization** (Dropout, Weight Decay)
4. **Architecture** (plus de couches, plus de filtres)
5. **Transfer Learning** (fine-tuning d'un modèle pré-entraîné)

In [ ]:
# TODO: Fine-tuning d'un modèle pré-entraîné (alternative au CNN from scratch)
def create_finetuned_model(model_name='resnet50', num_classes=2, freeze_ratio=0.8):
    """
    Crée un modèle pré-entraîné pour fine-tuning.
    
    Args:
        model_name: 'resnet50', 'vgg16', etc.
        num_classes: Nombre de classes (2 pour notre cas)
        freeze_ratio: Proportion des couches à geler (0.8 = 80% gelées)
    """
    if model_name == 'resnet50':
        model = models.resnet50(pretrained=True)
        # Remplacer la dernière couche
        model.fc = nn.Linear(model.fc.in_features, num_classes)
        
        # Geler les premières couches
        layers = list(model.children())
        freeze_until = int(len(layers) * freeze_ratio)
        for i, layer in enumerate(layers[:freeze_until]):
            for param in layer.parameters():
                param.requires_grad = False
    
    # TODO: Ajouter VGG16, DenseNet si besoin
    
    return model

# model_ft = create_finetuned_model('resnet50').to(DEVICE)

---
# 📊 SECTION 3c: Comparaison CNN vs Shallow Methods
---

## 🎯 Objectif
Comparer les performances de:
1. **CNN Baseline** (entraîné from scratch)
2. **CNN Fine-tuné** (transfer learning)
3. **Features pré-entraînées + Shallow Classifiers**

In [ ]:
# TODO: Créer un tableau de comparaison
def compare_models(models_results):
    """
    Compare les performances de plusieurs modèles.
    
    Args:
        models_results: Dict {'model_name': {'accuracy': ..., 'recall': ..., 'f1': ...}}
    
    Returns:
        DataFrame avec la comparaison
    """
    df = pd.DataFrame(models_results).T
    df = df.sort_values('f1', ascending=False)
    
    # Afficher avec style
    styled = df.style.background_gradient(cmap='Greens', subset=['accuracy', 'recall', 'f1'])
    
    return styled

# Exemple d'utilisation:
# results = {
#     'CNN Baseline': {'accuracy': 0.85, 'precision': 0.83, 'recall': 0.80, 'f1': 0.82},
#     'ResNet50 + SVM': {'accuracy': 0.90, 'precision': 0.89, 'recall': 0.88, 'f1': 0.89},
#     'ResNet50 + XGBoost': {'accuracy': 0.92, 'precision': 0.91, 'recall': 0.90, 'f1': 0.91},
#     'VGG16 + RandomForest': {'accuracy': 0.88, 'precision': 0.86, 'recall': 0.85, 'f1': 0.86},
# }
# compare_models(results)

In [ ]:
# Visualisation graphique de la comparaison
def plot_model_comparison(results):
    """
    Graphique de comparaison des modèles.
    """
    df = pd.DataFrame(results).T.reset_index()
    df = df.rename(columns={'index': 'Model'})
    
    fig, ax = plt.subplots(figsize=(10, 6))
    
    x = np.arange(len(df))
    width = 0.2
    
    ax.bar(x - width, df['accuracy'], width, label='Accuracy', color='#2E86AB')
    ax.bar(x, df['recall'], width, label='Recall', color='#F18F01')
    ax.bar(x + width, df['f1'], width, label='F1-Score', color='#4CAF50')
    
    ax.set_ylabel('Score')
    ax.set_title('Comparaison des Modèles')
    ax.set_xticks(x)
    ax.set_xticklabels(df['Model'], rotation=45, ha='right')
    ax.legend()
    ax.set_ylim(0, 1)
    ax.axhline(y=0.9, color='red', linestyle='--', alpha=0.5, label='Objectif 90%')
    
    plt.tight_layout()
    plt.show()

# plot_model_comparison(results)

---
# 🗳️ SECTION 3d: Ensemble - Vote Majoritaire
---

## 💡 Concept

Combiner les prédictions de plusieurs modèles:
- Si la majorité prédit "Défaut" → Prédiction finale = "Défaut"
- Confiance = nombre de votes pour la classe gagnante / total des modèles

### Avantages:
- Plus robuste qu'un seul modèle
- Réduit le risque de faux négatifs
- Capture différentes "perspectives" sur les données

In [ ]:
class EnsemblePredictor:
    """
    Prédicteur ensemble avec vote majoritaire.
    """
    
    def __init__(self):
        self.models = {}  # {'model_name': (model, feature_extractor, scaler)}
        self.cnn_models = {}  # CNN models qui prédisent directement
    
    def add_shallow_model(self, name, model, feature_extractor, scaler):
        """
        Ajoute un modèle shallow (SVM, XGBoost, etc.)
        """
        self.models[name] = {
            'model': model,
            'extractor': feature_extractor,
            'scaler': scaler
        }
    
    def add_cnn_model(self, name, model):
        """
        Ajoute un modèle CNN (prédit directement sur l'image)
        """
        self.cnn_models[name] = model
    
    def predict(self, image_tensor):
        """
        Prédit avec vote majoritaire.
        
        Args:
            image_tensor: Tensor de l'image (1, 3, 224, 224)
        
        Returns:
            dict: {
                'prediction': 0 ou 1,
                'confidence': float,
                'votes': {'defective': n, 'non_defective': m},
                'model_predictions': {'model_name': pred, ...}
            }
        """
        predictions = {}
        
        # TODO: Prédictions des CNN
        for name, model in self.cnn_models.items():
            model.eval()
            with torch.no_grad():
                output = model(image_tensor.to(DEVICE))
                pred = output.argmax(dim=1).item()
                predictions[name] = pred
        
        # TODO: Prédictions des shallow models
        for name, components in self.models.items():
            extractor = components['extractor']
            scaler = components['scaler']
            model = components['model']
            
            # Extraire features
            extractor.eval()
            with torch.no_grad():
                features = extractor(image_tensor.to(DEVICE))
                features = features.cpu().numpy().flatten().reshape(1, -1)
            
            # Normaliser
            features_scaled = scaler.transform(features)
            
            # Prédire
            pred = model.predict(features_scaled)[0]
            predictions[name] = pred
        
        # Vote majoritaire
        votes = list(predictions.values())
        defective_votes = sum(votes)
        non_defective_votes = len(votes) - defective_votes
        
        final_pred = 1 if defective_votes > non_defective_votes else 0
        confidence = max(defective_votes, non_defective_votes) / len(votes)
        
        return {
            'prediction': final_pred,
            'class_name': 'Defective' if final_pred == 1 else 'Non-Defective',
            'confidence': confidence,
            'votes': {
                'defective': defective_votes,
                'non_defective': non_defective_votes
            },
            'model_predictions': predictions
        }

# Exemple d'utilisation:
# ensemble = EnsemblePredictor()
# ensemble.add_cnn_model('CNN_Baseline', cnn_model)
# ensemble.add_shallow_model('ResNet50_SVM', svm_model, resnet_extractor, scaler)
# result = ensemble.predict(image_tensor)
# print(result)

---
# 🎯 SECTION 4: Entraînement des Classifiers
---

## 💡 Approche Hybride
Deep Features (CNN) + Shallow Classifiers (ML)

### Classifiers à tester:
1. **SVM** - Support Vector Machine
2. **Random Forest** - Ensemble d'arbres
3. **XGBoost** - Gradient Boosting

In [ ]:
# Normalisation des features (IMPORTANT!)
scaler = StandardScaler()

# TODO: Normaliser vos features
# X_train_scaled = scaler.fit_transform(X_train)
# X_test_scaled = scaler.transform(X_test)  # Utiliser le même scaler!

In [ ]:
# TODO: Entraîner un SVM
def train_svm(X_train, y_train):
    """
    Entraîne un SVM avec kernel RBF.
    
    Hint: sklearn.svm.SVC
    Paramètres importants: C, kernel, gamma
    """
    # VOTRE CODE ICI
    model = None
    return model

# TODO: Entraîner un Random Forest
def train_random_forest(X_train, y_train):
    """
    Entraîne un Random Forest.
    
    Paramètres importants: n_estimators, max_depth
    """
    # VOTRE CODE ICI
    model = None
    return model

---
# 📏 SECTION 5: Évaluation du Modèle
---

## 💡 Métriques importantes

Dans le contexte industriel:
- **Recall** est CRITIQUE (ne pas manquer de défauts)
- **F1-Score** pour la balance globale

In [ ]:
def evaluate_model(model, X_test, y_test):
    """
    Évalue un modèle et affiche les métriques.
    """
    # Prédictions
    y_pred = model.predict(X_test)
    
    # Métriques
    accuracy = accuracy_score(y_test, y_pred)
    precision = precision_score(y_test, y_pred)
    recall = recall_score(y_test, y_pred)
    f1 = f1_score(y_test, y_pred)
    
    print("=" * 50)
    print("RÉSULTATS D'ÉVALUATION")
    print("=" * 50)
    print(f"Accuracy:  {accuracy:.4f}")
    print(f"Precision: {precision:.4f}")
    print(f"Recall:    {recall:.4f}  ⬅️ IMPORTANT!")
    print(f"F1-Score:  {f1:.4f}")
    print("=" * 50)
    
    # Matrice de confusion
    cm = confusion_matrix(y_test, y_pred)
    plt.figure(figsize=(8, 6))
    sns.heatmap(cm, annot=True, fmt='d', cmap='Blues',
                xticklabels=['Non-Défaut', 'Défaut'],
                yticklabels=['Non-Défaut', 'Défaut'])
    plt.xlabel('Prédit')
    plt.ylabel('Réel')
    plt.title('Matrice de Confusion')
    plt.show()
    
    return {'accuracy': accuracy, 'precision': precision, 'recall': recall, 'f1': f1}

# Usage:
# results = evaluate_model(svm_model, X_test_scaled, y_test)

---
# ⚙️ SECTION 6: Optimisation des Hyperparamètres (Jour 2)
---

## 💡 Grid Search
Recherche exhaustive des meilleurs hyperparamètres.

Grid Search pour SVM, RandomForest, XGBoost

In [ ]:
# TODO: Grid Search pour SVM
def optimize_svm(X_train, y_train):
    """
    Trouve les meilleurs hyperparamètres pour SVM.
    
    Hint: GridSearchCV avec cv=5 (5-fold cross-validation)
    """
    param_grid = {
        'C': [0.1, 1, 10, 100],
        'kernel': ['rbf', 'linear'],
        'gamma': ['scale', 'auto']
    }
    
    # VOTRE CODE ICI
    # Hint: GridSearchCV(SVC(), param_grid, cv=5, scoring='f1')
    
    best_model = None
    best_params = None
    
    return best_model, best_params

# best_svm, best_params = optimize_svm(X_train_scaled, y_train)
# print(f"Meilleurs paramètres: {best_params}")

---
# 🌐 Interface Streamlit (Jour 2-4)
---

## 💡 Structure de l'application

```python
# app.py
import streamlit as st
from PIL import Image

st.title("🔍 Détection de Défauts")

# Upload
uploaded = st.file_uploader("Upload image", type=['jpg', 'png'])

if uploaded:
    image = Image.open(uploaded)
    st.image(image, caption="Image uploadée")
    
    if st.button("Analyser"):
        # 1. Prédiction
        prediction, confidence = model.predict(image)
        
        # 2. Afficher résultat
        if prediction == 1:
            st.error(f"⚠️ DÉFAUT DÉTECTÉ - Confiance: {confidence:.1%}")
        else:
            st.success(f"✅ OK - Confiance: {confidence:.1%}")
        
        
```

### Lancer l'application:
```bash
streamlit run app.py
```

---
# 💾 Sauvegarde des Modèles
---

In [ ]:
import joblib
import pickle

def save_pipeline(models_dir, classifier, scaler, cbir_data):
    """
    Sauvegarde tous les composants nécessaires.
    """
    models_dir = Path(models_dir)
    models_dir.mkdir(exist_ok=True)
    
    # Classifier
    joblib.dump(classifier, models_dir / 'classifier.pkl')
    
    # Scaler
    joblib.dump(scaler, models_dir / 'scaler.pkl')
    
    # CBIR data
    with open(models_dir / 'cbir_signatures.pkl', 'wb') as f:
        pickle.dump(cbir_data, f)
    
    print(f"✅ Modèles sauvegardés dans {models_dir}")

# Usage:
# save_pipeline(
#     MODELS_DIR,
#     classifier=best_svm,
#     scaler=scaler,
#     cbir_data={'features': X_train, 'labels': y_train, 'paths': paths_train}
# )

---
# ✅ CHECKLIST FINALE
---

## Jour 1
- [ ] EDA complète
- [ ] Dataset et DataLoader fonctionnels
- [ ] Feature extraction avec CNN
- [ ] Baseline classifier entraîné
- [ ] Métriques évaluées

## Jour 2
- [ ] Grid Search implémenté
- [ ] Meilleurs hyperparamètres trouvés
- [ ] Interface Streamlit de base


---

